Spark版本为2.0.0,Python版本为3.5.2,Jupyter notebook server版本为4.2.1

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.
<img src="http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png">
<img src="http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png">

# **Word Count Lab: Building a word count application**

This lab will build on the techniques covered in the Spark tutorial to develop a simple word count application.  The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data.  In this lab, we will write code that calculates the most common words(**最常见的词**) in the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100)(**莎士比亚全集**) retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page)(**古登堡计划是一个以自由的和电子化的形式,基于互联网,大量提供版权过期而进入公有领域书籍的一项协作计划.**).  This could also be scaled to larger applications, such as finding the most common words in Wikipedia.

** During this lab we will cover: **
* *Part 1:* Creating a base DataFrame and performing operations
* *Part 2:* Counting with Spark SQL and DataFrames
* *Part 3:* Finding unique words and a mean value
* *Part 4:* Apply word count to a file

Note that for reference, you can look up the details of the relevant methods in [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.sql).
#### ** Part 1: Creating a base DataFrame and performing operations **
In this part of the lab, we will explore creating a base DataFrame with **`sqlContext.createDataFrame`** and using DataFrame operations to count words.
** (1a) Create a DataFrame **

We'll start by generating a base DataFrame by using a Python list of tuples and the `sqlContext.createDataFrame` method.  Then we'll print out the type and schema of the DataFrame.  The Python API has several examples for using the [`createDataFrame` method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.createDataFrame).

In [1]:
wordsDF = sqlContext.createDataFrame([('cat',), ('elephant',), ('rat',), ('rat',), ('cat',)], ['word'])
wordsDF.show()
print(type(wordsDF))
wordsDF.printSchema()

+--------+
|    word|
+--------+
|     cat|
|elephant|
|     rat|
|     rat|
|     cat|
+--------+

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- word: string (nullable = true)



** (1b) Using DataFrame functions to add an 's' **

Let's create a new DataFrame from `wordsDF` by performing an operation that adds an 's' to each word.  To do this, we'll call the [`select` DataFrame function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select) and pass in a column that has the recipe for adding an 's' to our existing column.  To generate this `Column` object you should use the [`concat` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat) found in the [`pyspark.sql.functions` module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions).  Note that `concat` takes in two or more string columns and returns a single string column.  In order to pass in a constant or literal value like 's', you'll need to wrap that value with the [`lit` column function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lit).
>literal value:在计算机科学中,字面值(literal暂且翻译成该意思)是代表源代码中固定值的符号。几乎所有的编程语言都有代表原子值的符号。字面值通常用来初始化变量，例如：1是整数字面值，"cat"是字符串字面值。<br/>
>int a = 1;<br/>
>String s = "cat";

Please replace `<FILL IN>` with your solution.  After you have created `pluralDF` you can run the next cell which contains two tests.  If you implementation is correct it will print `1 test passed` for each test.

This is the general form that exercises will take.  Exercises will include an explanation of what is expected, followed by code cells where one cell will have one or more `<FILL IN>` sections.  The cell that needs to be modified will have `# TODO: Replace <FILL IN> with appropriate code` on its first line.  Once the `<FILL IN>` sections are updated and the code is run, the test cell can then be run to verify the correctness of your solution.  The last code cell before the next markdown section will contain the tests.

> Note:
> Make sure that the resulting DataFrame has one column which is named 'word'.

In [2]:
from pyspark.sql.functions import concat,lit
pluraDF = wordsDF.select(concat(wordsDF.word,lit('s')).alias('word'))
pluraDF.show()

+---------+
|     word|
+---------+
|     cats|
|elephants|
|     rats|
|     rats|
|     cats|
+---------+



** (1c) Length of each word **

Now use the SQL `length` function to find the number of characters in each word.  The [`length` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.length) is found in the `pyspark.sql.functions` module.

In [3]:
from pyspark.sql.functions import length
pluraLengthsDF = pluraDF.select(length(pluraDF.word).alias('length'))
pluraLengthsDF.show()

+------+
|length|
+------+
|     4|
|     9|
|     4|
|     4|
|     4|
+------+



#### ** Part 2: Counting with Spark SQL and DataFrames **
Now, let's count the number of times a particular word appears in the 'word' column. There are multiple ways to perform the counting, but some are much less efficient than others.

A naive approach would be to call `collect` on all of the elements and count them in the driver program. While this approach could work for small datasets, we want an approach that will work for any size dataset including terabyte- or petabyte-sized datasets. In addition, performing all of the work in the driver program is slower than performing it in parallel in the workers. For these reasons, we will use **data parallel operations**.

** (2a) Using `groupBy` and `count` **

Using DataFrames, we can preform aggregations by grouping the data using the [`groupBy` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) on the DataFrame.  Using `groupBy` returns a [`GroupedData` object](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) and we can use the functions available for `GroupedData` to aggregate the groups.  For example, we can call `avg` or `count` on a `GroupedData` object to obtain the average of the values in the groups or the number of occurrences in the groups, respectively.

To find the counts of words, group by the words and then use the [`count` function](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count) to find the number of times that words occur.

In [4]:
wordCountsDF = (pluraDF
                .groupBy('word')
                .agg({'word':"count"}))
wordCountsDF.show()

+---------+-----------+
|     word|count(word)|
+---------+-----------+
|     rats|          2|
|elephants|          1|
|     cats|          2|
+---------+-----------+



#### ** Part 3: Finding unique words and a mean value **
** (3a) Unique words **

Calculate the number of unique words in `wordsDF`.  You can use other DataFrames that you have already created to make this easier.

In [5]:
uniqueWordsCount = wordCountsDF.count()
print(uniqueWordsCount)

3


** (3b) Means of groups using DataFrames **

Find the mean number of occurrences of words in `wordCountsDF`.

You should use the [`mean` GroupedData method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.mean) to accomplish this.  Note that when you use `groupBy` you don't need to pass in any columns.  A call without columns just prepares the DataFrame so that aggregation functions like `mean` can be applied.

In [6]:
averageCount = (wordCountsDF
               .groupBy()
               .mean()
               .first()[0])
print(averageCount)

1.6666666666666667


#### ** Part 4: Apply word count to a file **
In this section we will finish developing our word count application.  We'll have to build the `wordCount` function, deal with real world problems like capitalization and punctuation(字母大写和标点符号), load in our data source, and compute the word count on the new data.

** (4a) The `wordCount` function **

First, define a function for word counting.  You should reuse the techniques that have been covered in earlier parts of this lab.  This function should take in a DataFrame that is a list of words like `wordsDF` and return a DataFrame that has all of the words and their associated counts.

In [7]:
def wordCount(wordListDF):
    """Creates a DataFrame with word counts.

    Args:
        wordListDF (DataFrame of str): A DataFrame consisting of one string column called 'word'.

    Returns:
        DataFrame of (str, int): A DataFrame containing 'word' and 'count' columns.
    """
    return wordListDF.groupBy("word").agg({'word':'count'})
wordCount(wordsDF).show()

+--------+-----------+
|    word|count(word)|
+--------+-----------+
|     rat|          2|
|     cat|          2|
|elephant|          1|
+--------+-----------+



** (4b) Capitalization and punctuation(大小写和标点符号) **

Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of(不依赖…的；不受…支配的) their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces(前导或尾随空格) on a line should be removed.

Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [regexp_replace](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_replace) module to remove any text that is not a letter, number, or space. If you are unfamiliar with regular expressions, you may want to review [this tutorial](https://developers.google.com/edu/python/regular-expressions) from Google.  Also, [this website](https://regex101.com/#python) is  a great resource for debugging your regular expression.

You should also use the `trim` and `lower` functions found in [pyspark.sql.functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions).

> Note that you shouldn't use any RDD operations or need to create custom user defined functions (udfs) to accomplish this task

In [8]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
def removePunctuation(column):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should be
        eliminated(消除；排除)(e.g. it's becomes its).  Leading and trailing spaces should 
        be removed after punctuation is removed.

    Args:
        column (Column): A Column containing a sentence(句子).

    Returns:
        Column: A Column named 'sentence' with clean-up operations applied.
    """
    return lower(trim(regexp_replace(column, '([^a-zA-Z0-9])', " ")))

sentenceDF = sqlContext.createDataFrame([('Hi, you!',),
                                         (' No under_score!',),
                                         (' *      Remove punctuation then spaces  * ',)],
                                        ['sentence'])
sentenceDF.show(truncate=False)
(sentenceDF
.select(removePunctuation(col('sentence')))
.show(truncate=False))

+------------------------------------------+
|sentence                                  |
+------------------------------------------+
|Hi, you!                                  |
| No under_score!                          |
| *      Remove punctuation then spaces  * |
+------------------------------------------+

+--------------------------------------------------------+
|lower(trim(regexp_replace(sentence, ([^a-zA-Z0-9]),  )))|
+--------------------------------------------------------+
|hi  you                                                 |
|no under score                                          |
|remove punctuation then spaces                          |
+--------------------------------------------------------+



** (4c) Load a text file **

For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into a DataFrame, we use the `sqlContext.read.text()` method. We also apply the recently defined `removePunctuation()` function using a `select()` transformation to strip out the punctuation and change all text to lower case.  Since the file is large we use `show(15)`, so that we only print 15 lines.

In [9]:
fileName = "datasets/shakespeare.txt"
shakespeareDFTemp = sqlContext.read.text(fileName)
print(shakespeareDFTemp.count())
shakespeareDFTemp.take(5)

124788


[Row(value='The Project Gutenberg EBook of The Complete Works of William Shakespeare, by'),
 Row(value='William Shakespeare'),
 Row(value=''),
 Row(value='This eBook is for the use of anyone anywhere at no cost and with'),
 Row(value='almost no restrictions whatsoever.  You may copy it, give it away or')]

In [10]:
shakespeareDF = shakespeareDFTemp.select(removePunctuation(col('value')).alias('word'))
shakespeareDF.show(5,truncate=False)

+----------------------------------------------------------------------------+
|word                                                                        |
+----------------------------------------------------------------------------+
|the project gutenberg ebook of the complete works of william shakespeare  by|
|william shakespeare                                                         |
|                                                                            |
|this ebook is for the use of anyone anywhere at no cost and with            |
|almost no restrictions whatsoever   you may copy it  give it away or        |
+----------------------------------------------------------------------------+
only showing top 5 rows



** (4d) Words from lines **

Before we can use the `wordcount()` function, we have to address two issues with the format of the DataFrame:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines or words.

Apply a transformation that will split each 'sentence' in the DataFrame by its spaces, and then transform from a DataFrame that contains lists of words into a DataFrame with each word in its own row.  To accomplish these two tasks you can use the `split` and `explode` functions found in [pyspark.sql.functions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions).

Once you have a DataFrame with one word per row you can apply the [DataFrame operation `where`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where) to remove the rows that contain ''.

> Note that `shakeWordsDF` should be a DataFrame with one column named `word`.

In [11]:
from pyspark.sql.functions import split, explode
shakeWordsDF = shakespeareDF.select(explode(split(shakespeareDF.word,"[ ]+")).alias("word")).where(length("word") != 0)
shakeWordsDF.show()
shakeWordsDFCount = shakeWordsDF.count()
print (shakeWordsDFCount)

+-----------+
|       word|
+-----------+
|        the|
|    project|
|  gutenberg|
|      ebook|
|         of|
|        the|
|   complete|
|      works|
|         of|
|    william|
|shakespeare|
|         by|
|    william|
|shakespeare|
|       this|
|      ebook|
|         is|
|        for|
|        the|
|        use|
+-----------+
only showing top 20 rows

932356


** (4e) Count the words **

We now have a DataFrame that is only words.  Next, let's apply the `wordCount()` function to produce a list of word counts. We can view the first 20 words by using the `show()` action; however, we'd like to see the words in descending order of count, so we'll need to apply the [`orderBy` DataFrame method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy) to first sort the DataFrame that is returned from `wordCount()`.

You'll notice that many of the words are common English words. These are called stopwords. In a later lab, we will see how to eliminate them from the results.

In [12]:
from pyspark.sql.functions import desc
topWordsAndCountsDF = wordCount(shakeWordsDF).orderBy(desc("count(word)"))
topWordsAndCountsDF.show()

+----+-----------+
|word|count(word)|
+----+-----------+
| the|      27843|
| and|      26847|
|   i|      22538|
|  to|      19882|
|  of|      18307|
|   a|      14800|
| you|      13928|
|  my|      12490|
|that|      11563|
|  in|      11183|
|  is|       9808|
|   d|       8961|
| not|       8760|
| for|       8358|
|with|       8066|
|  me|       7778|
|  it|       7750|
|   s|       7734|
|  be|       7146|
|this|       6900|
+----+-----------+
only showing top 20 rows

